In [1]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
from torch import nn 
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset


from utils import *
from dataset import *
from preprocess import *
from wrapper import *
from models import *

torch.cuda.is_available()
device = torch.device('cpu')

In [2]:
df = pd.read_csv('../data/data-org/train.csv', sep='\t').set_index('id')
corpus = df[df.label == 0].drop(columns=['label'])

In [5]:
ds = SimpleDataset(
    corpus, 
    model_name='KoichiYasuoka/chinese-bert-wwm-ext-upos', 
    test=True, 
    eda=False, 
    device=device, 
)
ds.prepare_dataset()

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("KoichiYasuoka/chinese-bert-wwm-ext-upos")
model = AutoModelForTokenClassification.from_pretrained("KoichiYasuoka/chinese-bert-wwm-ext-upos")

def feedforward(
    self, 
    ds:DatasetWithAuxiliaryEmbeddings, 
    checkpoints:List[str], 
    device:torch.device, 
    raw_outputs:bool=True, 
    output_probabilities:bool=False, 
    majority_vote=False, 
) -> np.ndarray:
    output_tensors = []
    output_sequence_logits = []

    if 'cuda' in device.type:
        self.model.cuda()

    logits = []
    sequence_logits = []
    dataloader = DataLoader(ds.dataset['train'].with_format('torch'), batch_size=16)

    for batch in dataloader:
        inputs = {k:v.to(device) for k,v in batch.items()
                if k in ds.tokenizer.model_input_names or k == 'auxiliary_input_ids'}
        with torch.no_grad():
            output = self.model(**inputs)
